In [19]:
import os
import data_processing_tool as dpt
from datetime import timedelta, date, datetime
import argparse

import torch,os,torchvision
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset,random_split
from torchvision import datasets, models, transforms
from PIL import Image
import time
import xarray as xr
from sklearn.model_selection import StratifiedShuffleSplit

file_ACCESS_dir="/g/data/ub7/access-s1/hc/raw_model/atmos/pr/daily/"
file_BARRA_dir="/g/data/ma05/BARRA_R/analysis/acum_proc"
ensemble=['e01','e02']
# ensemble=['e01','e02','e03','e04','e05','e06','e07','e08','e09','e10','e11']

leading_time=217
leading_time_we_use=31


init_date=date(1970, 1, 1)
start_date=date(1990, 1, 1)
end_date=date(1990,12,31) #if 929 is true we should substract 1 day
dates=[start_date + timedelta(x) for x in range((end_date - start_date).days + 1)]


In [2]:
parser = argparse.ArgumentParser(description='BARRA_R and ACCESS-S!')
def set_parser():
    parser.add_argument('--debug', action='store_true',
                        help='Enables debug mode')
    parser.add_argument('--template', default='.',
                        help='You can set various templates in option.py')

    # Hardware specifications
    parser.add_argument('--n_threads', type=int, default=3,
                        help='number of threads for data loading')
    parser.add_argument('--cpu', action='store_true',
                        help='use cpu only')
    parser.add_argument('--n_GPUs', type=int, default=2,
                        help='number of GPUs')
    parser.add_argument('--seed', type=int, default=1,
                        help='random seed')

    # Data specifications
    parser.add_argument('--domain', type=list, 
                        default=[111.85, 155.875, -44.35, -9.975],
                        help='dataset directory')    
    
    
    parser.add_argument('--file_ACCESS_dir', type=str, 
                        default="F:/climate/access-s1/pr/daily/",
    
                        help='dataset directory')
    parser.add_argument('--file_BARRA_dir', type=str, 
                        default="C:/Users/JIA059/barra/",
                        help='dataset directory')
    parser.add_argument('--nine2nine', type=bool, 
                        default=True,
                        help='whether rainfall acculate from 9am to 9am')
    parser.add_argument('--date_minus_one', type=int, 
                        default=1,
                        help='whether rainfall acculate from yesterday(1)/today(0) 9am to tody/tomorrow 9am')
    
    
    parser.add_argument('--dir_demo', type=str, default='../test',
                        help='demo image directory')
#     parser.add_argument('--data_train', type=str, default='BARRA_R',
#                         help='train dataset name')
#     parser.add_argument('--data_test', type=str, default='DIV2K',
#                         help='test dataset name')
    parser.add_argument('--benchmark_noise', action='store_true',
                        help='use noisy benchmark sets')
    parser.add_argument('--n_train', type=int, default=800,
                        help='number of training set')
    parser.add_argument('--n_val', type=int, default=10,
                        help='number of validation set')
    parser.add_argument('--offset_val', type=int, default=800,
                        help='validation index offest')
    parser.add_argument('--ext', type=str, default='sep',
                        help='dataset file extension')
    parser.add_argument('--scale', default='4',
                        help='super resolution scale')
    parser.add_argument('--patch_size', type=int, default=96,
                        help='output patch size')
    #??????????????????????????????????????????????????
    parser.add_argument('--rgb_range', type=int, default=300,
                        help='maximum value of RGB')
    parser.add_argument('--n_colors', type=int, default=1,
                        help='number of color channels to use')
    parser.add_argument('--noise', type=str, default='.',
                        help='Gaussian noise std.')
    parser.add_argument('--chop', action='store_true',
                        help='enable memory-efficient forward')

    # Model specifications
    parser.add_argument('--model', default='RCAN',
                        help='model name')

    parser.add_argument('--act', type=str, default='relu',
                        help='activation function')
    parser.add_argument('--pre_train', type=str, default='.',
                        help='pre-trained model directory')
    parser.add_argument('--extend', type=str, default='.',
                        help='pre-trained model directory')
    parser.add_argument('--n_resblocks', type=int, default=16,
                        help='number of residual blocks')
    parser.add_argument('--n_feats', type=int, default=64,
                        help='number of feature maps')
    parser.add_argument('--res_scale', type=float, default=1,
                        help='residual scaling')
    parser.add_argument('--shift_mean', default=True,
                        help='subtract pixel mean from the input')
    parser.add_argument('--precision', type=str, default='single',
                        choices=('single', 'half'),
                        help='FP precision for test (single | half)')

    # Training specifications
    parser.add_argument('--reset', action='store_true',
                        help='reset the training')
    parser.add_argument('--test_every', type=int, default=1000,
                        help='do test per every N batches')
    parser.add_argument('--epochs', type=int, default=3000,
                        help='number of epochs to train')
    parser.add_argument('--batch_size', type=int, default=16,
                        help='input batch size for training')
    parser.add_argument('--split_batch', type=int, default=1,
                        help='split the batch into smaller chunks')
    parser.add_argument('--self_ensemble', action='store_true',
                        help='use self-ensemble method for test')
    parser.add_argument('--test_only', action='store_true',
                        help='set this option to test the model')
    parser.add_argument('--gan_k', type=int, default=1,
                        help='k value for adversarial loss')

    # Optimization specifications
    parser.add_argument('--lr', type=float, default=1e-4,
                        help='learning rate')
    parser.add_argument('--lr_decay', type=int, default=200,
                        help='learning rate decay per N epochs')
    parser.add_argument('--decay_type', type=str, default='step',
                        help='learning rate decay type')
    parser.add_argument('--gamma', type=float, default=0.5,
                        help='learning rate decay factor for step decay')
    parser.add_argument('--optimizer', default='ADAM',
                        choices=('SGD', 'ADAM', 'RMSprop'),
                        help='optimizer to use (SGD | ADAM | RMSprop)')
    parser.add_argument('--momentum', type=float, default=0.9,
                        help='SGD momentum')
    parser.add_argument('--beta1', type=float, default=0.9,
                        help='ADAM beta1')
    parser.add_argument('--beta2', type=float, default=0.999,
                        help='ADAM beta2')
    parser.add_argument('--epsilon', type=float, default=1e-8,
                        help='ADAM epsilon for numerical stability')
    parser.add_argument('--weight_decay', type=float, default=0,
                        help='weight decay')

    # Loss specifications
    parser.add_argument('--loss', type=str, default='1*L1',
                        help='loss function configuration')
    parser.add_argument('--skip_threshold', type=float, default='1e6',
                        help='skipping batch that has large error')

    # Log specifications
    parser.add_argument('--save', type=str, default='RCAN',
                        help='file name to save')
    parser.add_argument('--load', type=str, default='.',
                        help='file name to load')
    parser.add_argument('--resume', type=int, default=0,
                        help='resume from specific checkpoint')
    parser.add_argument('--print_model', action='store_true',
                        help='print model')
    parser.add_argument('--save_models', action='store_true',
                        help='save all intermediate models')
    parser.add_argument('--print_every', type=int, default=100,
                        help='how many batches to wait before logging training status')
    parser.add_argument('--save_results', action='store_true',
                        help='save output results')

    # New options
    parser.add_argument('--n_resgroups', type=int, default=10,
                        help='number of residual groups')
    parser.add_argument('--reduction', type=int, default=16,
                        help='number of feature maps reduction')
    parser.add_argument('--testpath', type=str, default='../test/DIV2K_val_LR_our',
                        help='dataset directory for testing')
    parser.add_argument('--testset', type=str, default='Set5',
                        help='dataset name for testing')
    parser.add_argument('--degradation', type=str, default='BI',
                        help='degradation model: BI, BD')

    args = parser.parse_args(args=[])
    # args = parser.parse_args()
#     template.set_template(args)

    args.scale = list(map(lambda x: int(x), args.scale.split('+')))
    
    if args.epochs == 0:
        args.epochs = 1e8

    for arg in vars(args):
        if vars(args)[arg] == 'True':
            vars(args)[arg] = True
        elif vars(args)[arg] == 'False':
            vars(args)[arg] = False
    return args

args=set_parser()
# args.template.find("DDBPN")


In [42]:

# domain = [111.975, 156.275, -44.525, -9.975]

file_ACCESS_dir="F:/climate/access-s1/pr/daily/"#"/g/data/ub7/access-s1/hc/raw_model/atmos/pr/daily/"
file_BARRA_dir="F:/climate/barra/"
from sklearn.model_selection import train_test_split
class ACCESS_BARRA_v1(Dataset):
    '''
    scale is size(hr)=size(lr)*scale
    version_1_documention: the data we use is raw data that store at NCI
    '''
    def __init__(self,start_date=date(1990, 1, 1),end_date=date(1990,12 , 31),regin="AUS",transform=None,train=True,args=args):
        self.file_BARRA_dir = args.file_BARRA_dir
        self.file_ACCESS_dir = args.file_ACCESS_dir
        
        self.transform = transform
        self.start_date = start_date
        self.end_date = end_date
        
        self.scale = args.scale[0]
        self.regin = regin
        
#         if regin=="AUS":
#             self.shape=(314,403,1,1)
#             self.domain=[111.975, 156.275, -44.525, -9.975]
#         else:
#             self.shape=(768,1200,1,1)
                
        self.dates = self.date_range(start_date, end_date)
        
        
        self.filename_list=self.get_filename_with_time_order(args.file_ACCESS_dir)

        
        
    def __len__(self):
        return len(self.filename_list)
    

    def date_range(self,start_date, end_date):
        """This function takes a start date and an end date as datetime date objects.
        It returns a list of dates for each date in order starting at the first date and ending with the last date"""
        return [start_date + timedelta(x) for x in range((end_date - start_date).days + 1)]

    
    def get_filename_with_no_time_order(self,rootdir):
        '''get filename first and generate label '''
        _files = []
        list = os.listdir(rootdir) #列出文件夹下所有的目录与文件
        for i in range(0,len(list)):
            path = os.path.join(rootdir,list[i])
            if os.path.isdir(path):
                _files.extend(self.get_filename_with_no_time_order(path))
            if os.path.isfile(path):
                if path[-3:]==".nc":
                    _files.append(path)
        return _files
    
    def get_filename_with_time_order(self,rootdir):
        '''get filename first and generate label '''
        _files = []
        for en in ensemble:
            for date in dates:
                filename="da_pr_"+date.strftime("%Y%m%d")+"_"+en+".nc"
                access_path=rootdir+en+"/"+"da_pr_"+date.strftime("%Y%m%d")+"_"+en+".nc"
#                 print(access_path)
                if os.path.exists(access_path):
                    for i in range(leading_time_we_use):
                        path=[access_path]
                        
#                         barra_path=file_BARRA_dir+"/accum_prcp-an-spec-PT0H-BARRA_R-v1-"+((date+timedelta(i)).strftime("%Y%m%d"))
                        barra_date=date+timedelta(i)
#                         self.data_dir+date.strftime('%m')+"/accum_prcp-an-spec-PT0H-BARRA_R-v1-"\
#                         +date.strftime('%Y%m%d')+"T"+enum[i]+"Z.nc"
                        path.append(barra_date)
                        path.append(i)
#                         print(path)
                        _files.append(path)
    
    #最后去掉第一行，然后shuffle
        if args.nine2nine and args.date_minus_one==1:
            del _files[0]
        return _files

    

        
    def __getitem__(self, idx):
        '''
        from filename idx get id
        return lr,hr
        '''
        #read_data filemame[idx]
        access_filename,date_for_BARRA,time_leading=self.filename_list[idx]
#         print(type(date_for_BARRA))
#         low_filename,high_filename,time_leading=self.filename_list[idx]

        data_low=dpt.read_access_data(access_filename,idx=time_leading)
        train_data_raw=dpt.map_aust(data_low,domain=args.domain,xrarray=False)
        
#         domain = [train_data.lon.data.min(), train_data.lon.data.max(), train_data.lat.data.min(), train_data.lat.data.max()]
#         print(domain)

        data_high=dpt.read_barra_data_fc(self.file_BARRA_dir,date_for_BARRA,nine2nine=True)
        label=dpt.map_aust(data_high,domain=args.domain,xrarray=False)#,domain=domain)
        
        train_data=dpt.interp_tensor_2d(train_data_raw,(78,100))
        
#         print(train_data_raw.shape,label.shape)
#         print(train_data_raw.shape[0]*4,train_data_raw.shape[1]*4)
#         print(label.shape[0]/4,label.shape[1]/4)
#         print(type(train_data))

        return train_data*86400,label

    




<class 'int'>
(array([[ 2.21364753e-01,  2.14401749e-01,  2.03532614e-01, ...,
         2.85297778e-01,  2.05935444e-01,  1.61991234e-01],
       [ 2.81123591e-01,  2.59208869e-01,  2.24326623e-01, ...,
         1.19194396e-01,  7.48954300e-02,  4.98846184e-02],
       [ 2.71889752e-01,  2.47006442e-01,  2.07226892e-01, ...,
         4.10489250e-03, -4.70327843e-03, -9.62451234e-03],
       ...,
       [ 1.80859736e+01,  1.87983415e+01,  1.97787941e+01, ...,
         1.46401248e-01,  3.86206028e-02, -3.49697404e-02],
       [ 1.88613717e+01,  2.12084169e+01,  2.47479245e+01, ...,
         1.66117193e-01,  1.42921013e-01,  1.25619606e-01],
       [ 1.36590325e+01,  1.67238418e+01,  2.14907781e+01, ...,
         2.01601797e-01,  3.69652067e-01,  4.79752792e-01]]), array([[1.17721516e-01, 1.03651812e-01, 1.03662699e-01, ...,
        1.62397884e+00, 1.62766278e+00, 1.84940102e+00],
       [1.06871511e-01, 9.73634421e-02, 2.13461184e-01, ...,
        1.06457847e+00, 9.68259435e-01, 1.148118

In [46]:
# ACCESS_BARRA(file_access_dir,file_BARRA_dir).filename_list
data_set=ACCESS_BARRA_v1(args=args)
# print(type(len(data_set)))
# for i in data_set.filename_list:
#     print(i)
print(data_set.filename_list[0])

['F:/climate/access-s1/pr/daily/e01/da_pr_19900101_e01.nc', datetime.date(1990, 1, 2), 1]


In [43]:
train_data,test_data=random_split(data_set,[int(len(data_set)*0.8),len(data_set)-int(len(data_set)*0.8)])

In [44]:
print(len(train_data))
print(test_data[0])

2231
(array([[0.44458577, 0.50948099, 0.61956215, ..., 3.68310559, 3.01365702,
        2.56756946],
       [0.94797117, 0.87947229, 0.78684752, ..., 2.06559918, 2.20666708,
        2.29902429],
       [1.22405495, 1.22373623, 1.23805812, ..., 0.83642166, 1.58333905,
        2.07518677],
       ...,
       [0.46879649, 0.41306425, 0.32625484, ..., 0.94026668, 0.85529634,
        0.80764294],
       [0.50658755, 0.51541998, 0.53222996, ..., 1.71173885, 1.48478547,
        1.34669183],
       [0.47485307, 0.61775193, 0.84823173, ..., 2.21317071, 2.01842253,
        1.90304081]]), array([[ 4.24336898,  4.09088373,  3.92935878, ...,  0.72233323,
         1.03680729,  1.26014083],
       [ 4.29919159,  3.74399622,  3.74353586, ...,  0.75500704,
         0.95867663,  1.2478332 ],
       [ 4.00387657,  3.94200559,  3.85101742, ...,  0.77019322,
         0.90085714,  0.87491499],
       ...,
       [12.09644707, 11.16179435, 10.15246886, ..., 19.40720924,
        18.65332822, 18.81883149],
    